In [ ]:
pip install kfp

In [1]:
import json
import kfp
import kfp.dsl as dsl
import kfp.components as comp
from kfp.components import OutputPath
import pandas as pd
from google.cloud import storage # save the model to GCS
from datetime import datetime, date, timedelta
import ast
import json
import http.client
import mimetypes
import pandas as pd
import requests

# API key (TawabG)
API_KEY = 'goldapi-rnmc2ukhqm7vnk-io'

In [ ]:
# replace this value with the value of the KFP host name

# Kubeflow Pipeline van TawabG (te vinden bij: GCP -> AI Platform -> Pipelines -> Open Pipelines Dashboard)
KFP_HOST_NAME = 'https://47ccb329d434cbca-dot-europe-west1.pipelines.googleusercontent.com/'

# change caching policy of KFP run (useful in debug/dev mode)
disable_cache = False
# year from which to use the financial data
year_from = 1990
# number of latest days to holdout for validation
holdout_days = 30
# number of random iterations for random search hyperparameter optimization
random_iterations = 3
# Number of trees in random forest
n_estimators = [150, 250, 300, 400]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [5, 10, 25]
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True]

client = kfp.Client(host=KFP_HOST_NAME)

In [28]:
# TODO TAWAB
    # 1. download from .csv file
    # 2. check last row date
    # 3. check today's date
    # 4. for all the days in between last row date and today's date, call from API. 
    # 5. add new data from API call to existing dataframe
    # 6. save new dataframe as .CSV

In [2]:
# gold_dataset = pd.read_csv("gs://gold-price-prediction/gold_dataset.csv")  
gold_dataset = pd.read_csv("gold_dataset.csv")  
gold_dataset.head()

,date,timestamp,metal,exchange,currency,price,prev_close_price,ch,chp
0,2020-01-01T10:30:00.000Z,1.577875e+12,XAU,LBMA,EUR,1355.7059,1355.7059,0.0000,0.0000
1,2020-01-02T10:30:00.000Z,1.577961e+12,XAU,LBMA,EUR,1358.4830,1360.6718,-2.1889,-0.1611
2,2020-01-03T10:30:00.000Z,1.578047e+12,XAU,LBMA,EUR,1388.1762,1364.0890,24.0872,1.7352
3,2020-01-04T10:30:00.000Z,1.578134e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
4,2020-01-05T10:30:00.000Z,1.578220e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000


In [8]:
def calc_new_dates():
    date_today = datetime.today().strftime('%Y-%m-%d')
    gold_dataset['date'] = pd.to_datetime(gold_dataset['date'], errors='coerce')
    gold_dataset['date'] = gold_dataset['date'].dt.date
    last_date=gold_dataset.iloc[-1]['date']
    date_today = datetime.strptime(date_today, '%Y-%m-%d')
#     last_date = datetime.strptime(last_date, '%Y-%m-%d')
#     print(date_today)
#     print(last_date)
#     print(type(date_today))
#     print(type(last_date))
    
    sdate = date_today   # start date
    edate = last_date   # end date

    list_dates = pd.date_range(edate,sdate-timedelta(days=0),freq='d').strftime("%Y%m%d").tolist()
    list_dates_int = list(map(int, list_dates))
    
    return list_dates_int
    

In [10]:
list_dates_int = calc_new_dates()
list_dates_int

[20201119,
 20201120,
 20201121,
 20201122,
 20201123,
 20201124,
 20201125,
 20201126,
 20201127,
 20201128,
 20201129]

In [11]:
def make_api_call(API_KEY, date):
    
    conn = http.client.HTTPSConnection("www.goldapi.io")
    payload = ''
    headers = {
      'x-access-token': API_KEY,
      'Content-Type': 'application/json'
    }
    conn.request("GET", f"/api/XAU/EUR/{date}", payload, headers)
    res = conn.getresponse()

    return res.read()

In [5]:
def download_data():
    
    result = []
    for dates in calc_new_dates():
        data = make_api_call(API_KEY, dates)
        result.append(data.decode("utf-8"))
    result
    final_result = [json.loads(items) for items in result]
    return final_result

In [12]:
result = download_data()

In [13]:
gold_dataset_new = pd.DataFrame.from_dict(result)
gold_dataset_new = gold_dataset_new.drop(labels='error', axis=1)
gold_dataset_new = gold_dataset_new.dropna()
gold_dataset_new['date'] = pd.to_datetime(gold_dataset_new['date'], errors='coerce')
gold_dataset_new['date'] = gold_dataset_new['date'].dt.date
gold_dataset_new

,date,timestamp,metal,exchange,currency,price,prev_close_price,ch,chp
0,2020-11-19,1.605782e+12,XAU,LBMA,EUR,1569.8107,1586.5450,-16.7343,-1.0660
1,2020-11-20,1.605868e+12,XAU,LBMA,EUR,1573.8009,1565.7085,8.0924,0.5142
2,2020-11-21,1.605955e+12,XAU,LBMA,EUR,1573.8009,1573.8009,0.0000,0.0000
3,2020-11-22,1.606041e+12,XAU,LBMA,EUR,1573.8009,1573.8009,0.0000,0.0000
4,2020-11-23,1.606127e+12,XAU,LBMA,EUR,1566.0869,1568.7757,-2.6888,-0.1717
5,2020-11-24,1.606214e+12,XAU,LBMA,EUR,1532.3220,1570.8386,-38.5166,-2.5136
6,2020-11-25,1.606300e+12,XAU,LBMA,EUR,1521.0681,1529.1001,-8.0320,-0.5280
7,2020-11-26,1.606387e+12,XAU,LBMA,EUR,1525.0840,1519.7899,5.2941,0.3471
8,2020-11-27,1.606473e+12,XAU,LBMA,EUR,1516.5660,1522.2698,-5.7037,-0.3761
9,2020-11-28,1.606559e+12,XAU,LBMA,EUR,1516.5660,1516.5660,0.0000,0.0000


In [14]:
#gold_dataset.append(gold_dataset_new)
gold_dataset = gold_dataset.append(gold_dataset_new, ignore_index = True) 
gold_dataset              

,date,timestamp,metal,exchange,currency,price,prev_close_price,ch,chp
0,2020-01-01,1.577875e+12,XAU,LBMA,EUR,1355.7059,1355.7059,0.0000,0.0000
1,2020-01-02,1.577961e+12,XAU,LBMA,EUR,1358.4830,1360.6718,-2.1889,-0.1611
2,2020-01-03,1.578047e+12,XAU,LBMA,EUR,1388.1762,1364.0890,24.0872,1.7352
3,2020-01-04,1.578134e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
4,2020-01-05,1.578220e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
329,2020-11-24,1.606214e+12,XAU,LBMA,EUR,1532.3220,1570.8386,-38.5166,-2.5136
330,2020-11-25,1.606300e+12,XAU,LBMA,EUR,1521.0681,1529.1001,-8.0320,-0.5280
331,2020-11-26,1.606387e+12,XAU,LBMA,EUR,1525.0840,1519.7899,5.2941,0.3471
332,2020-11-27,1.606473e+12,XAU,LBMA,EUR,1516.5660,1522.2698,-5.7037,-0.3761


In [17]:
# TODO Tawab
    # 1. Load new dataframe
    # 2. create new features dataframe with new acquired data
    # 3. Save new DF to .csv (google storage)

In [15]:
gold_dataset = gold_dataset.set_index('date')
gold_dataset

,timestamp,metal,exchange,currency,price,prev_close_price,ch,chp
date,,,,,,,,
2020-01-01,1.577875e+12,XAU,LBMA,EUR,1355.7059,1355.7059,0.0000,0.0000
2020-01-02,1.577961e+12,XAU,LBMA,EUR,1358.4830,1360.6718,-2.1889,-0.1611
2020-01-03,1.578047e+12,XAU,LBMA,EUR,1388.1762,1364.0890,24.0872,1.7352
2020-01-04,1.578134e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
2020-01-05,1.578220e+12,XAU,LBMA,EUR,1388.1762,1388.1762,0.0000,0.0000
...,...,...,...,...,...,...,...,...
2020-11-24,1.606214e+12,XAU,LBMA,EUR,1532.3220,1570.8386,-38.5166,-2.5136
2020-11-25,1.606300e+12,XAU,LBMA,EUR,1521.0681,1529.1001,-8.0320,-0.5280
2020-11-26,1.606387e+12,XAU,LBMA,EUR,1525.0840,1519.7899,5.2941,0.3471


In [16]:
def create_features_dataframe(dataset):
    
    gold_features_df = pd.DataFrame()
    
    average_days_window_closing_price = [5, 30]
    for window in average_days_window_closing_price:
        gold_features_df['Mean__'+str(window)+'_days'] = dataset['price'].rolling(window).mean().shift(periods=1)
        gold_features_df['Std__'+str(window)+'_days'] = dataset['price'].rolling(window).std().shift(periods=1)
        gold_features_df['Max__'+str(window)+'_days'] = dataset['price'].rolling(window).max().shift(periods=1)
        gold_features_df['Min__'+str(window)+'_days'] = dataset['price'].rolling(window).min().shift(periods=1)
        gold_features_df['Min__'+str(window)+'_days'] = dataset['price'].rolling(window).sum().shift(periods=1)
        
    # get day of the week
    dataset['date'] = dataset.index
    dataset['day_of_week'] = dataset['date'].apply(lambda x: x.strftime("%A"))

    # get quarter
    dataset['month'] = dataset['date'].apply(lambda x: x.strftime("%B"))
    
    gold_features_df = pd.concat([gold_features_df, pd.get_dummies(dataset['day_of_week']), pd.get_dummies(dataset['month'])], 1)

    return gold_features_df

features_df = create_features_dataframe(gold_dataset)
features_df.tail()

,Mean__5_days,Std__5_days,Max__5_days,Min__5_days,Mean__30_days,Std__30_days,Max__30_days,Min__30_days,Friday,Monday,...,August,February,January,July,June,March,May,November,October,September
date,,,,,,,,,,,,,,,,,,,,,
2020-11-24,1571.46006,3.465178,1573.8009,7857.3003,1600.921700,23.238375,1647.2692,48027.6510,0,0,...,0,0,0,0,0,0,0,1,0,0
2020-11-25,1563.96232,18.000116,1573.8009,7819.8116,1598.367940,26.331244,1647.2692,47951.0382,0,0,...,0,0,0,0,0,0,0,1,0,0
2020-11-26,1553.41576,24.914772,1573.8009,7767.0788,1595.574040,29.830159,1647.2692,47867.2212,0,0,...,0,0,0,0,0,0,0,1,0,0
2020-11-27,1543.67238,24.471672,1573.8009,7718.3619,1592.493287,32.167874,1647.2692,47774.7986,1,0,...,0,0,0,0,0,0,0,1,0,0
2020-11-28,1532.22540,19.794647,1566.0869,7661.1270,1589.592250,34.937170,1647.2692,47687.7675,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
features_df.to_csv(r'dynamic_features_df.csv', index = False)

In [ ]:
def create_models():
    # Romar